In [2]:
import numpy as np
import seaborn as sns
import glob
import os
import matplotlib.pyplot as plt
import tifffile
from skimage import color
from skimage import img_as_ubyte
from pystackreg import StackReg
from matplotlib import animation
from IPython.display import HTML
import json
import cv2
import statsmodels.api as sm
import pandas as pd
from skimage.draw import polygon2mask, polygon_perimeter
import xml.etree.ElementTree as ET
import multiprocess as mp
from PIL import Image, ImageDraw

In [7]:
def read_results(path_pattern, metadata_path):
    meta = pd.read_csv(metadata_path, sep='\t')
    
    dsets = []
    for path in (glob.glob('results8/*/vf.csv'):
        directory = os.path.dirname(path)
        print(directory)
        parts = directory.split(os.sep)
        sample = parts[1]
        #stain = parts[2]

        data = pd.read_csv(path)
        data['sample'] = sample
        #data['stain'] = stain
        data['genotype'] = meta[(meta['Animal']) == int(sample)]['Genotype'].values[0]
        dsets.append(data)

    all_data = pd.concat(dsets, ignore_index=True)
    all_data['vm'] = np.abs(np.sqrt(all_data['vx']**2 + all_data['vy']**2))
    return all_data

def get_animal_genotype(animal, metadata_path='metadataHET.txt'):
    meta = pd.read_csv(metadata_path, sep='\t')
    return meta[meta['Animal'] == animal]['Genotype'].values[0]
    
def poly_area(x, y):
    # https://stackoverflow.com/a/30408825
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def poly_to_px_coords(x, y, scale=1):
    x = x / scale
    y = y / scale
    return np.swapaxes(np.array([x, y]), 0, 1)
    
def poly_to_mask(x, y, shape, scale=1):
    coords = poly_to_px_coords(x, y, scale)
    msk = polygon2mask((shape[1], shape[0]), coords)
    msk = np.flipud(np.rot90(msk))
    return msk

def draw_poly(img, coords, c=None):
    msk = polygon_perimeter(coords[:, 0], coords[:, 1], shape=img.shape)
    if img.shape == 3:
        if c is None:
            c = (255, 0, 0) # Red
        # RGB image
        img[msk[0], msk[1], 0] = c[0]
        img[msk[0], msk[1], 1] = c[1]
        img[msk[0], msk[1], 2] = c[2]
    else:
        # grayscale image
        if c is None:
            c = 255 # White
        img[msk[0], msk[1]] = c
    return img


def load_images(sample, t=None):
    base_dir = r'/Users/garvitgupta/Downloads/front-tracking-main/data_Twist_CKO+_Sm22a-Cre'
    image_paths = sorted(glob.glob(os.path.join(base_dir, sample,'processed', '*.tif')))
    
    if t is None:
        images = []
        for p in image_paths:
            images.append(__load_img(p))
        return np.array(images)
    else:
        p = image_paths[t]
        return __load_img(p)

__imcache = {}
def __load_img(path):
    if path not in __imcache:
        im = tifffile.imread(path)
        im = color.rgb2gray(im)
        im = np.asarray(img_as_ubyte(im))
        __imcache[path] = im
    
    return __imcache[path]

__scalecache = {}
def read_scale(sample):
    key = (sample)
    if key not in __scalecache:
        base_dir = r'/Users/garvitgupta/Downloads/front-tracking-main/data_Twist_CKO+_Sm22a-Cre'
        meta_path = os.path.join(base_dir, sample,'*.tif')
        first = glob.glob(meta_path)[0]
        first_meta = os.path.join(base_dir, sample,'.Metadata', '{}.cal.xml'.format(os.path.basename(first)))

        ns = 'http://schemas.datacontract.org/2004/07/LeicaMicrosystems.DataEntities.V3_2'
        tree = ET.parse(first_meta)
        root = tree.getroot()
        res = tree.find(".//{"+ns+"}XMetresPerPixel").text
        
        __scalecache[key] = float(res) * 1e3
    return __scalecache[key]
    
def save_video(images, dest_dir, basename='video', codec='MP4V', fps=6):
    foutrcc = cv2.VideoWriter_fourcc(*codec)
    width = images.shape[2]
    height = images.shape[1]
    
    if codec in ['MP4V']:
        ext = 'mp4'
    elif codec in ['XVID']:
        ext = 'avi'
    
    dest_gray = os.path.join(dest_dir, '{}.gray.{}'.format(basename, ext))
    video_gray  = cv2.VideoWriter(dest_gray, foutrcc, fps, (width, height), isColor=False)
    
    dest_color = os.path.join(dest_dir, '{}.color.{}'.format(basename, ext))
    video_color = cv2.VideoWriter(dest_color, foutrcc, fps, (width, height), isColor=True)
    
    for i in range(images.shape[0]):
        video_gray.write(images[i])
        video_color.write(cv2.applyColorMap(images[i], cv2.COLORMAP_JET))
    video_gray.release()
    video_color.release()
    
def save_color_video(images, dest_dir, basename='video', codec='MP4V', fps=6):
    foutrcc = cv2.VideoWriter_fourcc(*codec)
    width = images.shape[2]
    height = images.shape[1]
    
    if codec in ['MP4V']:
        ext = 'mp4'
    elif codec in ['XVID']:
        ext = 'avi'
    
    os.makedirs(dest_dir, exist_ok=True)
    dest = os.path.join(dest_dir, '{}.{}'.format(basename, ext))
    video = cv2.VideoWriter(dest, foutrcc, fps, (width, height), isColor=True)
    
    for i in range(images.shape[0]):
        video.write(images[i])
    video.release()
    
def mask_worker(t, fi, group, shape, scale):
    msk = poly_to_mask(group['x'], group['y'], shape, scale)
    return t, fi, msk

In [8]:
all_data = read_results('results8\*\vf.csv', 'metadataHET.txt')
all_data

results8/12
results8/127
results8/75
results8/76
results8/77
results8/79
results8/83
results8/88
results8/9


,t,fi,x,y,vx,vy,wx,wy,th,sample,genotype,vm
0,120,1,0.067429,7.745919,NaN,NaN,NaN,NaN,NaN,12,HET,NaN
1,120,1,0.071513,7.743405,NaN,NaN,NaN,NaN,NaN,12,HET,NaN
2,120,1,0.075755,7.741238,NaN,NaN,NaN,NaN,NaN,12,HET,NaN
3,120,1,0.080102,7.739451,NaN,NaN,NaN,NaN,NaN,12,HET,NaN
4,120,1,0.084507,7.738062,NaN,NaN,NaN,NaN,NaN,12,HET,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1983621,3600,477,15.772007,8.534232,NaN,NaN,NaN,NaN,NaN,9,HET,NaN
1983622,3600,477,15.772676,8.531448,NaN,NaN,NaN,NaN,NaN,9,HET,NaN
1983623,3600,477,15.773688,8.528654,NaN,NaN,NaN,NaN,NaN,9,HET,NaN
1983624,3600,477,15.775032,8.525862,NaN,NaN,NaN,NaN,NaN,9,HET,NaN


In [9]:
# Create videos of front areas over time

for sample in all_data['sample'].unique():

        subset = all_data[(all_data['sample'] == sample)]
        fronts = subset.groupby(['t', 'fi'])

        scale = read_scale(sample)
        imgs = load_images(sample)
        pil_images = []
        pil_ctxs = []
        for i in range(imgs.shape[0]):
            im = Image.fromarray(np.zeros(imgs[i].shape, dtype='bool'))
            imd = ImageDraw.Draw(im)
            pil_images.append(im)
            pil_ctxs.append(imd)
        
        
        for (t, fi), group in fronts:
            idx = int(t/60)
            im = pil_ctxs[idx]
            coords = poly_to_px_coords(group['x'], group['y'], scale)
            im.polygon([(p[0], p[1]) for p in coords], fill=1)
        
        imgs2 = np.zeros((imgs.shape[0], imgs.shape[1], imgs.shape[2], 3), dtype='uint8')
        for i in range(imgs.shape[0]):
            b = np.array(Image.fromarray(imgs[i]).convert("RGB"))
            f = cv2.applyColorMap(imgs[i], cv2.COLORMAP_JET)
            m = np.array(pil_images[i])
            
            mask = np.zeros_like(b)
            for mi in range(3):
                mask[:,:,mi] = m.copy()
            
            imgs2[i] = np.where(mask, f, b)
            
            
        save_color_video(imgs2, 'front_movies_areas', '{}'.format(sample))

/var/folders/3n/fcpvb3h559x4r03p9bdnxsy40000gn/T/ipykernel_13686/688907998.py:75: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  im = color.rgb2gray(im)
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: 

In [ ]:
plt.imshow(imgs2[0])

In [ ]:
save_video(np.ma.filled(msk_imgs, 0), '.')

In [10]:
# Create videos of front lines over time

for sample in all_data['sample'].unique():

        subset = all_data[(all_data['sample'] == sample)]
        fronts = subset.groupby(['t', 'fi'])

        scale = read_scale(sample)
        imgs = load_images(sample)
        pil_images = []
        pil_ctxs = []
        for i in range(imgs.shape[0]):
            im = Image.fromarray(imgs[i]).convert("RGB")
            imd = ImageDraw.Draw(im)
            pil_images.append(im)
            pil_ctxs.append(imd)
        
        
        for (t, fi), group in fronts:
            idx = int(t/60)
            im = pil_ctxs[idx]
            coords = poly_to_px_coords(group['x'], group['y'], scale)
            im.line([(p[0], p[1]) for p in coords], fill=(255, 0, 0), width=10, joint='curve')
        
        imgs2 = np.zeros((imgs.shape[0], imgs.shape[1], imgs.shape[2], 3), dtype='uint8')
        for i in range(imgs.shape[0]):
            imgs2[i] = np.array(pil_images[i])
            
        save_color_video(imgs2, '.', '{}'.format(sample))

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Open

In [ ]:
plt.imshow(imgs[])

In [ ]:
save_color_video(imgs2, '.')